In [ ]:
!pip install datasets
!pip install hazm
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.1 MB/s 
     |████████████████████████████████| 141 kB 83.1 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
     |████████████████████████████████| 115 kB 78.7 MB/s 
     |████████████████████████████████| 212 kB 74.8 MB/s 
     |████████████████████████████████| 596 kB 65.8 MB/s 
     |████████████████████████████████| 127 kB 77.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 4.1 MB/s 
     |██████████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import hazm

import pandas as pd
corpus = hazm.PersicaReader(csv_file='/content/drive/My Drive/persica.csv')
df=pd.DataFrame(corpus.docs())
df.drop(['id','title','date','time','category'],inplace=True,axis=1)

In [ ]:
df['category2']=df['category2'].astype('category')
df['labels']=df['category2'].cat.codes
df.head(5)
df.drop('category2',inplace=True,axis=1)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df["labels"]):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]
from datasets import Dataset
persica_train = Dataset.from_pandas(strat_train_set)
persica_test = Dataset.from_pandas(strat_test_set)


In [ ]:
  from transformers import AutoTokenizer
  import tensorflow as tf
  from transformers import TFAutoModelForSequenceClassification

In [ ]:
def train_model(train_dataset,test_dataset,model_cpkt):
  tokenizer =AutoTokenizer.from_pretrained(model_cpkt)
  def tokenize_function(example):
    return tokenizer(example['text'],padding='max_length',truncation=True)
  tokenized_dataset=train_dataset.map(tokenize_function,batched=True)
  tokenized_test=test_dataset.map(tokenize_function,batched=True)
  from transformers import DefaultDataCollator
  data_collator=DefaultDataCollator(return_tensors='tf')
  tf_train_dataset=tokenized_dataset.to_tf_dataset(
      columns=[  'input_ids', 'token_type_ids', 'attention_mask'],
      label_cols=['labels'],
      shuffle=False,
      collate_fn=data_collator,
      batch_size=8,
    )
  tf_test_dataset=tokenized_test.to_tf_dataset(
      columns=[  'input_ids', 'token_type_ids', 'attention_mask'],
      label_cols=['labels'],
      shuffle=False,
      collate_fn=data_collator,
      batch_size=8,
    )

  model=TFAutoModelForSequenceClassification.from_pretrained(model_cpkt, num_labels=11)
  return model,tf_train_dataset,tf_test_dataset

In [ ]:
model1,train1,test1=train_model(persica_train,persica_test,"HooshvareLab/bert-fa-zwnj-base")
model2,train2,test2=train_model(persica_train,persica_test,"bert-base-multilingual-uncased")
model3,train3,test3=train_model(persica_train,persica_test,'distilbert-base-multilingual-cased')

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_287']
You should probably TRAIN this model on a down-stream t

In [ ]:
model3,train3,test3=train_model(persica_train,persica_test,'distilbert-base-multilingual-cased')

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_267']
You should probably TRAIN this model on a down-stream t

In [ ]:
import tensorflow as tf
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=10000,
    decay_rate=0.9)
model1.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=tf.metrics.SparseCategoricalAccuracy(),
) 
model1.fit(train1,epochs=10)


Epoch 1/10
1100/1100 [==============================] - 667s 592ms/step - loss: 0.7494 - sparse_categorical_accuracy: 0.7777
Epoch 2/10
1100/1100 [==============================] - 651s 592ms/step - loss: 0.3990 - sparse_categorical_accuracy: 0.8849
Epoch 3/10
1100/1100 [==============================] - 651s 592ms/step - loss: 0.2867 - sparse_categorical_accuracy: 0.9157
Epoch 4/10
1100/1100 [==============================] - 651s 592ms/step - loss: 0.2171 - sparse_categorical_accuracy: 0.9374
Epoch 5/10
1100/1100 [==============================] - 651s 592ms/step - loss: 0.1741 - sparse_categorical_accuracy: 0.9499
Epoch 6/10
1100/1100 [==============================] - 651s 592ms/step - loss: 0.1284 - sparse_categorical_accuracy: 0.9628
Epoch 7/10
1100/1100 [==============================] - 651s 592ms/step - loss: 0.1023 - sparse_categorical_accuracy: 0.9693
Epoch 8/10
1100/1100 [==============================] - 651s 592ms/step - loss: 0.0951 - sparse_categorical_accuracy: 0.9750


In [ ]:
model1.evaluate(test1)

275/275 [==============================] - 61s 210ms/step - loss: 0.8329 - sparse_categorical_accuracy: 0.8532


[0.8328551054000854, 0.8531818389892578]

In [ ]:
model2.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=tf.metrics.SparseCategoricalAccuracy(),
)
model2.fit(train2,epochs=10)

Epoch 1/10
1100/1100 [==============================] - 681s 605ms/step - loss: 1.1013 - sparse_categorical_accuracy: 0.6621
Epoch 2/10
1100/1100 [==============================] - 666s 605ms/step - loss: 0.6192 - sparse_categorical_accuracy: 0.8184
Epoch 3/10
1100/1100 [==============================] - 667s 606ms/step - loss: 0.5451 - sparse_categorical_accuracy: 0.8425
Epoch 4/10
1100/1100 [==============================] - 667s 606ms/step - loss: 0.4929 - sparse_categorical_accuracy: 0.8569
Epoch 5/10
1100/1100 [==============================] - 667s 606ms/step - loss: 0.4315 - sparse_categorical_accuracy: 0.8727
Epoch 6/10
1100/1100 [==============================] - 667s 606ms/step - loss: 0.3827 - sparse_categorical_accuracy: 0.8867
Epoch 7/10
1100/1100 [==============================] - 667s 606ms/step - loss: 0.3685 - sparse_categorical_accuracy: 0.8950
Epoch 8/10
1100/1100 [==============================] - 667s 606ms/step - loss: 0.3250 - sparse_categorical_accuracy: 0.9075


In [ ]:
model2.evaluate(test2)

275/275 [==============================] - 61s 212ms/step - loss: 0.6404 - sparse_categorical_accuracy: 0.8382


[0.6403647065162659, 0.8381817936897278]

In [ ]:
model3.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=tf.metrics.SparseCategoricalAccuracy(),
)
model3.fit(train3,epochs=10)

Epoch 1/10
1100/1100 [==============================] - 351s 311ms/step - loss: 1.2646 - sparse_categorical_accuracy: 0.5927
Epoch 2/10
1100/1100 [==============================] - 342s 311ms/step - loss: 0.6546 - sparse_categorical_accuracy: 0.8015
Epoch 3/10
1100/1100 [==============================] - 342s 311ms/step - loss: 0.4728 - sparse_categorical_accuracy: 0.8574
Epoch 4/10
1100/1100 [==============================] - 342s 311ms/step - loss: 0.3834 - sparse_categorical_accuracy: 0.8902
Epoch 5/10
1100/1100 [==============================] - 342s 311ms/step - loss: 0.3293 - sparse_categorical_accuracy: 0.9051
Epoch 6/10
1100/1100 [==============================] - 342s 311ms/step - loss: 0.2704 - sparse_categorical_accuracy: 0.9201
Epoch 7/10
1100/1100 [==============================] - 342s 310ms/step - loss: 0.2413 - sparse_categorical_accuracy: 0.9329
Epoch 8/10
1100/1100 [==============================] - 342s 311ms/step - loss: 0.2133 - sparse_categorical_accuracy: 0.9412


In [ ]:
model3.evaluate(test3)

275/275 [==============================] - 30s 105ms/step - loss: 0.7006 - sparse_categorical_accuracy: 0.8491


[0.7006405591964722, 0.8490909337997437]